In [ ]:
# te model and predictions

In [1]:
import pandas as pd

In [2]:
stats = pd.read_csv("ffb_stats.csv")

In [3]:
feats = ['Player','Tm','Pos','Year','Pts','Tgt','Rec','Receiving Yds','Receiving TD']

In [4]:
ffb_te = stats[stats['Pos'] == 'TE']

In [5]:
te = ffb_te[feats]

In [6]:
predictors = ['Tgt','Rec','Receiving Yds','Receiving TD']

In [8]:
train = te[te['Year'] < 2022]
test = te[te['Year'] == 2022]

In [9]:
from sklearn.linear_model import Ridge

tem = Ridge(alpha=.1)

In [10]:
tem.fit(train[predictors], train["Pts"])

Ridge(alpha=0.1)

In [11]:
predictions = tem.predict(test[predictors])

In [12]:
predictions = pd.DataFrame(predictions, columns=["Pred Pts"], index=test.index)

In [13]:
comb = pd.concat([test[["Player", "Pts"]], predictions], axis=1)

In [14]:
comb

,Player,Pts,Pred Pts
348,Travis Kelce,201.0,203.050183
884,George Kittle,140.0,140.782033
1093,Taysom Hill,127.0,19.025459
1124,T.J. Hockenson,125.0,125.189475
1370,Mark Andrews,110.0,112.806894
...,...,...,...
6998,Richard Rodgers,0.0,-0.074565
7018,Stephen Anderson,0.0,0.379291
7019,Devin Asiasi,0.0,-0.008158
7022,Mason Schreck,0.0,0.079087


In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
mean_squared_error(comb["Pts"], comb["Pred Pts"])

101.0507634879643

In [17]:
import pickle

In [18]:
with open('tem.pkl', 'wb') as model_file:
    pickle.dump(tem, model_file)

In [19]:
te_proj = pd.read_csv('te_projections.csv')

In [20]:
del te_proj ['Proj Pts']

In [22]:
te_proj.columns

Index(['Player', 'Team', 'Tgt', 'Rec', 'Rec Yds', 'Rec TDs'], dtype='object')

In [23]:
te_proj.rename(columns={'Rec Yds': 'Receiving Yds'}, inplace=True)

In [24]:
te_proj.rename(columns={'Rec TDs': 'Receiving TD'}, inplace=True)

In [25]:
te_proj.rename(columns={'Team': 'Tm'}, inplace=True)

In [26]:
te_proj = te_proj.fillna(0)

In [27]:
te_pred = tem.predict(te_proj[predictors])

In [28]:
te_pred = pd.DataFrame(te_pred, columns=["Pred Pts"], index=te_proj.index)

In [29]:
te_comb = pd.concat([te_proj[["Player", "Tm"]], te_pred], axis=1)

In [30]:
te_comb = te_comb.sort_values("Pred Pts", ascending=False)
te_comb["Pred Rk"] = list(range(1,te_comb.shape[0]+1))

In [31]:
te_comb

,Player,Tm,Pred Pts,Pred Rk
0,Travis Kelce,KC,170.417383,1
1,Mark Andrews,BAL,128.543849,2
2,George Kittle,SF,113.422095,3
3,T.J. Hockenson,MIN,106.157414,4
5,Kyle Pitts,ATL,99.325048,5
...,...,...,...,...
78,Johnny Mundt,MIN,7.705934,79
79,Shane Zylstra,DET,7.698960,80
80,Jeremy Ruckert,NYJ,7.046660,81
81,Cole Turner,WAS,5.050413,82


In [32]:
te_comb.to_csv('proj_te.csv')